In [2]:
!pip install datasets

In [3]:
from datasets import load_dataset

In [6]:
# dataset = load_dataset("thunlp/few_rel")


In [7]:
# print(dataset)

In [8]:
import json

with open('pid2name.json') as f:
  relation_data = json.load(f)

In [9]:
relation_data

{'P2384': ['statement describes',
  'formalization of the statement contains a bound variable in this class'],
 'P2388': ['office held by head of the organization',
  'position of the head of this item'],
 'P2389': ['organization directed from the office or person',
  'No description defined'],
 'P2634': ['sitter',
  'person who posed during the creation of a work, whether or not that person is eventually depicted as oneself'],
 'P607': ['conflict',
  'battles, wars or other military engagements in which the person or item participated'],
 'P608': ['exhibition history',
  'exhibitions where the item is or was displayed'],
 'P609': ['terminus location', 'location of the terminus of a linear feature'],
 'P237': ['coat of arms', "subject's coat of arms"],
 'P2925': ['domain of saint or deity',
  'domain(s) which this saint or deity controls or protects'],
 'P927': ['anatomical location',
  'where in the body does this anatomical feature lie'],
 'P2319': ['elector',
  'people or other enti

In [38]:
import pandas as pd

# The 'lines=True' argument is crucial for large files where
# each line is a separate JSON object (JSON Lines format).
# Omit 'lines=True' if it's a single, monolithic JSON array.
try:
    df = pd.read_json('fewrel_train.json', lines=True)
except ValueError:
    # Try loading as a monolithic JSON file if lines=True fails
    df = pd.read_json('fewrel_train.json')

print("JSON loaded successfully into a DataFrame.")

JSON loaded successfully into a DataFrame.


In [11]:
df.head()

,P931,P4552,P140,P1923,P150,P6,P27,P449,P1435,P175,P1344,P39,P527,P740,P706,P84,P495,P123,P57,P22,P178,P241,P403,P1411,P135,P991,P156,P176,P31,P1877,P102,P1408,P159,P3373,P1303,P17,P106,P551,P937,P355,P710,P137,P674,P466,P136,P306,P127,P400,P974,P1346,P460,P86,P118,P264,P750,P58,P3450,P105,P276,P101,P407,P1001,P800,P131
0,"[{'tokens': ['Merpati', 'flight', '106', 'depa...","[{'tokens': ['It', 'is', 'approximately', '8',...","[{'tokens': ['In', '1689', ',', 'Konstanty', '...","[{'tokens': ['On', '26', 'August', '2016', ','...","[{'tokens': ['The', 'sailor', 'was', 'rescued'...","[{'tokens': ['Chester', 'Alan', 'Arthur', ',',...","[{'tokens': ['Vares', ':', 'Private', 'Eye', '...","[{'tokens': ['first', 'aired', 'against', 'fel...","[{'tokens': ['The', 'property', ',', 'includin...","[{'tokens': ['The', 'song', 'has', 'been', 're...","[{'tokens': ['At', 'the', '1992', 'Summer', 'O...","[{'tokens': ['Mitchell', 'Hepburn', ',', 'the'...","[{'tokens': ['In', '""', 'The', 'Telling', '""',...","[{'tokens': ['Similar', 'movements', 'concurre...","[{'tokens': ['The', 'island', 'lies', 'between...","[{'tokens': ['Pythius', 'and', 'Satyros', 'wer...","[{'tokens': ['Turn', 'It', 'Upside', 'Down', '...","[{'tokens': ['Financial', 'and', 'contractual'...","[{'tokens': ['Cheryl', 'Barnes', 'is', 'a', 's...","[{'tokens': ['Matilda', 'II', 'died', 'five', ...","[{'tokens': ['""', 'Resonance', 'of', 'Fate', '...","[{'tokens': ['Among', 'the', 'students', 'at',...","[{'tokens': ['The', 'Peel', 'watershed', 'drai...","[{'tokens': ['""', 'A', 'Star', 'is', 'Born', '...","[{'tokens': ['The', 'large', 'workshops', 'of'...","[{'tokens': ['Christine', 'Branstad', 'returne...","[{'tokens': ['It', 'is', 'the', 'fourth', 'ins...","[{'tokens': ['Notable', 'exhibitors', 'include...","[{'tokens': ['Logic', 'Express', 'was', 'a', '...","[{'tokens': ['The', 'Bread', 'of', 'Those', 'E...","[{'tokens': ['Rempel', 'became', 'involved', '...","[{'tokens': ['In', 'November', '2015', ',', 'W...","[{'tokens': ['It', 'can', 'be', 'viewed', 'at'...","[{'tokens': ['His', 'family', 'met', 'William'...","[{'tokens': ['He', 'began', 'studying', 'piano...","[{'tokens': ['Belgium', ''s', 'highest', 'poin...","[{'tokens': ['Cunlhat', 'was', 'the', 'birthpl...","[{'tokens': ['Wong', 'Fei', '-', 'hung', 'plan...","[{'tokens': ['Shortly', 'after', 'his', 'arriv...","[{'tokens': ['He', 'is', 'head', 'of', 'the', ...","[{'tokens': ['In', '2007', 'Yurii', 'Barybin',...","[{'tokens': ['The', 'South', 'Dakota', 'Cultur...","[{'tokens': ['Actress', 'Sally', 'Hawkins', 'w...","[{'tokens': ['The', 'ECHL', ''s', 'Kansas', 'C...","[{'tokens': ['Trailer', 'Bride', 'was', 'a', '...","[{'tokens': ['This', 'has', 'caused', 'the', '...","[{'tokens': ['Amityville', 'is', 'the', 'weste...","[{'tokens': ['For', 'instance', ',', 'Firebird...","[{'tokens': ['The', 'east', 'side', 'of', 'Gra...","[{'tokens': ['In', '1961', 'he', 'guided', 'Fi...","[{'tokens': ['While', 'he', 'was', 'clear', 'a...","[{'tokens': ['Rahman', 'made', 'a', 'clean', '...","[{'tokens': ['He', 'made', 'his', 'debut', 'in...","[{'tokens': ['The', 'Very', 'Air', 'We', 'Brea...","[{'tokens': ['Following', '""', 'The', 'Painted...","[{'tokens': ['""', 'Citizen', 'Khan', '""', 'is'...","[{'tokens': ['The', '1959', 'Los', 'Angeles', ...","[{'tokens': ['Polietes', 'is', 'a', 'genus', '...","[{'tokens': ['Links', 'Verlag', ',', '2008', '...","[{'tokens': ['It', 'was', 'first', 'described'...","[{'tokens': ['Japanese', 'video', 'sharing', '...","[{'tokens': ['In', 'April', '2014', ',', 'the'...","[{'tokens': ['Devaki', 'Nandan', 'Khatri', ','...","[{'tokens': ['Downing', 'House', 'is', 'a', 'h..."


In [17]:
df.shape

(1, 64)

In [21]:
len(df['P931'][0])

700

In [12]:
df.columns

Index(['P931', 'P4552', 'P140', 'P1923', 'P150', 'P6', 'P27', 'P449', 'P1435',
       'P175', 'P1344', 'P39', 'P527', 'P740', 'P706', 'P84', 'P495', 'P123',
       'P57', 'P22', 'P178', 'P241', 'P403', 'P1411', 'P135', 'P991', 'P156',
       'P176', 'P31', 'P1877', 'P102', 'P1408', 'P159', 'P3373', 'P1303',
       'P17', 'P106', 'P551', 'P937', 'P355', 'P710', 'P137', 'P674', 'P466',
       'P136', 'P306', 'P127', 'P400', 'P974', 'P1346', 'P460', 'P86', 'P118',
       'P264', 'P750', 'P58', 'P3450', 'P105', 'P276', 'P101', 'P407', 'P1001',
       'P800', 'P131'],
      dtype='object')

In [37]:
from google.colab import drive
drive.mount('/content/drive')
!cp -r zsbert_model /content/drive/MyDrive/


Mounted at /content/drive


In [39]:
count = 0
data_v1 = []
for i in df.columns:
  for j in range(len(df[i][0])):
    tokens = df[i][0][j]['tokens']
    head = df[i][0][j]['h']
    tail = df[i][0][j]['t']

    sentence  = ' '.join(tokens)

    data_v1.append({
        'p_id':i,
        'sentence':sentence,
        'head':head,
        'tail': tail
    })

    count += 1

print(count)

44800


In [40]:
data_v1[690:710]

[{'p_id': 'P931',
  'sentence': 'Rose Bay Water Airport ( IATA : RSE ) is an Australian water airport located in the Sydney suburb of Rose Bay , New South Wales .',
  'head': ['rose bay water airport', 'Q28219774', [[0, 1, 2, 3]]],
  'tail': ['sydney', 'Q3130', [[17]]]},
 {'p_id': 'P931',
  'sentence': 'Andrews County Airport is situated on 260 acres in Andrews , Texas , 1-mile northeast of the central business district , and contains three runways and one helipad .',
  'head': ['andrews county airport', 'Q4759072', [[0, 1, 2]]],
  'tail': ['andrews, texas', 'Q509541', [[9, 10, 11]]]},
 {'p_id': 'P931',
  'sentence': 'City Express was founded by Joseph Csumrik in 1971 as Otonabee Airways based at Peterborough Airport in Peterborough , Ontario .',
  'head': ['peterborough airport', 'Q905906', [[14, 15]]],
  'tail': ['peterborough, ontario', 'Q776930', [[17, 18, 19]]]},
 {'p_id': 'P931',
  'sentence': 'This were mostly out of Copenhagen Airport to Stavanger , Kristiansand , Göteborg Land

In [6]:
# count = 0
# data_v2 = []
# for example in data_v1[:10]:
#   sentence = example['sentence']
#   head = example['head'][0]
#   tail = example['tail'][0]

#   sentence = sentence.lower().replace(head,f"[E1] {head} [/E1]")
#   sentence = sentence.lower().replace(tail,f"[E2] {tail} [/E2]")

#   data_v2.append({
#       'sentence':sentence,
#       'head':head,
#       'tail': tail
#   })


#   count +=1

# print(count)

In [5]:
# data_v2

In [41]:

from transformers import AutoTokenizer
# from data_config import RAW_DATA, RELATION_TEMPLATES
import json
import random

# Define special tokens for entity marking as per your document (Figure 2)
ENTITY_START_TOKENS = {
    'head': '[E1]',
    'tail': '[E2]'
}
ENTITY_END_TOKENS = {
    'head': '[/E1]',
    'tail': '[/E2]'
}

# --- 2.1 Entity Marking Function ---
def insert_entity_markers(tokens, entity_h, entity_t):

    # FewRel data stores indices in the 3rd position (index 2) of the entity list.
    h_indices = entity_h[2]
    t_indices = entity_t[2]

    # Create a list of (index, token) tuples for sorting insertion points
    insertions = []

    # Entity H is Head ([E1], [/E1])
    # The start index is the start of the first span.
    h_start_idx = h_indices[0][0]
    # The end index is the token *after* the end of the last span.
    h_end_idx = h_indices[-1][-1] + 1

    insertions.append((h_start_idx, ENTITY_START_TOKENS['head']))
    insertions.append((h_end_idx, ENTITY_END_TOKENS['head']))

    # Entity T is Tail ([E2], [/E2])
    t_start_idx = t_indices[0][0]
    t_end_idx = t_indices[-1][-1] + 1

    insertions.append((t_start_idx, ENTITY_START_TOKENS['tail']))
    insertions.append((t_end_idx, ENTITY_END_TOKENS['tail']))

    \
    insertions.sort(key=lambda x: x[0], reverse=True)

    marked_tokens = list(tokens) # Copy the list

    for idx, token in insertions:
        # Check to ensure the insertion index is valid
        if 0 <= idx <= len(marked_tokens):
             marked_tokens.insert(idx, token)

    return " ".join(marked_tokens)



In [55]:
# for example in data_v1[:10]:
#   sentence = example['sentence']
#   head = example['head']
#   tail = example['tail']
#   marked_sentence = insert_entity_markers(senten, head, tail)

In [42]:
def demonstrate_marking(data_snippet):
    """Processes the sample data to demonstrate correct entity marking."""
    results = []
    print("--- Demonstration of Correct Entity Marking (Index-Based) ---")

    for i, example in enumerate(data_snippet):
        # 1. Tokenize the sentence by simple whitespace splitting (FewRel's convention)
        tokens = example['sentence'].split(' ')

        # 2. Get the marked sentence using the robust index-based function
        marked_sentence = insert_entity_markers(
            tokens,
            example['head'],
            example['tail']
        )

        result = {
            'p_id':example['p_id'],
            'original_sentence': example['sentence'],
            'head_entity': example['head'][0],
            'tail_entity': example['tail'][0],
            'marked_sentence': marked_sentence
        }
        results.append(result)

        # # Print a clean example for validation
        # print(f"\nExample {i+1}:")
        # print(f"  Head: {result['head_entity']} (Indices: {example['head'][2]})")
        # print(f"  Tail: {result['tail_entity']} (Indices: {example['tail'][2]})")
        # print(f"  Marked: {result['marked_sentence']}")

    return results


In [43]:
marked_sentence = demonstrate_marking(data_v1)

--- Demonstration of Correct Entity Marking (Index-Based) ---


In [54]:
marked_sentence

[{'p_id': 'P931',
  'original_sentence': 'Merpati flight 106 departed Jakarta ( CGK ) on a domestic flight to Tanjung Pandan ( TJQ ) .',
  'head_entity': 'tjq',
  'tail_entity': 'tanjung pandan',
  'marked_sentence': 'Merpati flight 106 departed Jakarta ( CGK ) on a domestic flight to [E2] Tanjung Pandan [/E2] ( [E1] TJQ [/E1] ) .'},
 {'p_id': 'P931',
  'original_sentence': 'The name was at one point changed to Nottingham East Midlands Airport so as to include the name of the city that is supposedly most internationally recognisable , mainly due to the Robin Hood legend .',
  'head_entity': 'east midlands airport',
  'tail_entity': 'nottingham',
  'marked_sentence': 'The name was at one point changed to [E2] Nottingham [/E2] [E1] East Midlands Airport [/E1] so as to include the name of the city that is supposedly most internationally recognisable , mainly due to the Robin Hood legend .'},
 {'p_id': 'P931',
  'original_sentence': 'It is a four - level stack interchange near Fort Lauderd

In [53]:
marked_sentence[23424]['p_id'] == 'P177'

False

In [18]:
with open('/content/pid2name.json') as f:
  relation_data = json.load(f)

In [19]:
relation_data

{'P2384': ['statement describes',
  'formalization of the statement contains a bound variable in this class'],
 'P2388': ['office held by head of the organization',
  'position of the head of this item'],
 'P2389': ['organization directed from the office or person',
  'No description defined'],
 'P2634': ['sitter',
  'person who posed during the creation of a work, whether or not that person is eventually depicted as oneself'],
 'P607': ['conflict',
  'battles, wars or other military engagements in which the person or item participated'],
 'P608': ['exhibition history',
  'exhibitions where the item is or was displayed'],
 'P609': ['terminus location', 'location of the terminus of a linear feature'],
 'P237': ['coat of arms', "subject's coat of arms"],
 'P2925': ['domain of saint or deity',
  'domain(s) which this saint or deity controls or protects'],
 'P927': ['anatomical location',
  'where in the body does this anatomical feature lie'],
 'P2319': ['elector',
  'people or other enti

In [78]:
template_map = {k:v[0] for k,v in relation_data.items()}

In [81]:
template_map['P931']

'place served by transport hub'

**Generating Entailments**

In [82]:
for ex in marked_sentence:
  relation_id = ex['p_id']
  print(relation_id)

P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931
P931


In [83]:
final_data = []

for ex in marked_sentence:
    relation_id = ex["p_id"]
    if relation_id in template_map:
        template = f"X {template_map[relation_id]} Y"
        final_data.append({
            "sentence": ex["marked_sentence"],
            "template": template,
            "label": 1
        })


In [84]:
final_data

[{'sentence': 'Merpati flight 106 departed Jakarta ( CGK ) on a domestic flight to [E2] Tanjung Pandan [/E2] ( [E1] TJQ [/E1] ) .',
  'template': 'X place served by transport hub Y',
  'label': 1},
 {'sentence': 'The name was at one point changed to [E2] Nottingham [/E2] [E1] East Midlands Airport [/E1] so as to include the name of the city that is supposedly most internationally recognisable , mainly due to the Robin Hood legend .',
  'template': 'X place served by transport hub Y',
  'label': 1},
 {'sentence': 'It is a four - level stack interchange near [E1] Fort Lauderdale - Hollywood International Airport [/E1] in [E2] Fort Lauderdale , Florida [/E2] .',
  'template': 'X place served by transport hub Y',
  'label': 1},
 {'sentence': "It is the main alternate of [E1] Jinnah International Airport [/E1] in [E2] Karachi [/E2] with a distance of about 350 \xa0 km/220 miles ; well under an hour 's flight time in turboprop aircraft .",
  'template': 'X place served by transport hub Y',
 

**Negative Entailments**

In [85]:
import random

relation_list = list(template_map.values())

for ex in marked_sentence:
    wrong_template = random.choice([t for t in relation_list if t != template_map[ex["p_id"]]])
    final_data.append({
        "sentence": ex["marked_sentence"],
        "template": f"X {wrong_template} Y",
        "label": 0
    })


In [86]:
len(final_data)

89600

**Saving data for finetuning**

In [4]:
# import json

# with open("zsbert_training_data.json", "w", encoding="utf-8") as f:
#     json.dump(final_data, f, indent=2, ensure_ascii=False)


In [2]:
# with open('/content/zsbert_training_data.json') as file:
#   data =

In [3]:
# !pip install datasets

In [2]:
# !head -n 5 /content/zsbert_training_data.json

In [10]:
# with open('/content/zsbert_training_data.json') as file:
#   data = json.load(file)

In [1]:
# data

In [11]:
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

# Load dataset
dataset = load_dataset("json", data_files="/content/zsbert_training_data.json")["train"]
dataset = dataset.shuffle(seed=42).train_test_split(test_size=0.2)

# Tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_fn(ex):
    return tokenizer(
        ex["sentence"],
        ex["template"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

tokenized = dataset.map(tokenize_fn, batched=True)
tokenized.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# Model



Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/71680 [00:00<?, ? examples/s]

Map:   0%|          | 0/17920 [00:00<?, ? examples/s]

In [12]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Training arguments
args = TrainingArguments(
    output_dir="zsbert_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01
)

# Trainer


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer
)



/tmp/ipython-input-2680076076.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [14]:
# Train and save
trainer.train()


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shayanumar5555 (shayanumar5555-international-islamic-university-islamabad) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.063800,0.057252
2,0.028500,0.044130


TrainOutput(global_step=8960, training_loss=0.06319269005741392, metrics={'train_runtime': 3348.5373, 'train_samples_per_second': 42.813, 'train_steps_per_second': 2.676, 'total_flos': 9429900224102400.0, 'train_loss': 0.06319269005741392, 'epoch': 2.0})

In [16]:
trainer.save_model("zsbert_model")
# tokenizer.save_pretrained("zsbert_model_tokenizer")

In [34]:
for rid, tmpl in relation_data.items():
  if tmpl[0] == 'crosses':
    print(rid)

P177


In [35]:
relation_data['P177']

['crosses',
 'obstacle (body of water, road, ...) which this bridge crosses over or this tunnel goes under']

In [57]:
import torch
import numpy as np
from transformers import BertForSequenceClassification, BertTokenizer

# --- 1. Define Model Path and Load Resources ---
MODEL_PATH = "zsbert_model"

# Load the fine-tuned model and tokenizer from the saved directory
model = BertForSequenceClassification.from_pretrained(MODEL_PATH)
tokenizer = BertTokenizer.from_pretrained(MODEL_PATH)

# Set device (use GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval() # Set model to evaluation mode

# --- 2. Define the Test Inputs (Sentence S and Relation Template R) ---

# Example 1: TRUE Relation (Expected Entailment / Label 1)
S_true = "[E1] Pakyong Airport [/E1] , a Greenfield project , is under construction southeast of [E2] Gangtok [/E2]."
R_true = "X place served by transport hub Y"

S_true = "[E1] Wilton Bridge [/E1] was a major crossing of the [E2] River Wye [/E2]"
R_true = "X crosses Y"

# Example 2: FALSE Relation (Expected Non-Entailment / Label 0)
S_false = "The [E1] Mona Lisa [/E1] was painted by [E2] Leonardo da Vinci [/E2]."
R_false = "X mountain range Y"


# ----------------------------------------------------
# Inference Function
# ----------------------------------------------------
def predict_relation(sentence, relation_template, model, tokenizer, device):
    # [cite_start]The ZS-BERT input format combines S and R: [CLS] S [SEP] R [SEP] [cite: 46, 64]

    # Clean entity markers (or use them for later attention masking if needed)
    text_a = sentence.replace("**", "")
    text_b = relation_template

    # Tokenizer handles the [CLS] and [SEP] tokens automatically for two input texts
    tokenized_input = tokenizer(
        text_a,
        text_b,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors="pt"
    )

    # Move input tensors to the correct device
    input_ids = tokenized_input['input_ids'].to(device)
    attention_mask = tokenized_input['attention_mask'].to(device)

    # Run inference with torch.no_grad()
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    # [cite_start]Apply softmax to get the probability distribution over the two classes (0=Non-Entailment, 1=Entailment) [cite: 48, 70]
    probabilities = torch.softmax(logits, dim=1).cpu().numpy()[0]

    # [cite_start]The final prediction is the class with the maximum probability [cite: 72]
    prediction = np.argmax(probabilities)

    # Probability of Entailment (y=1)
    entailment_prob = probabilities[1]

    return prediction, entailment_prob

# --- 3. Run and Print Predictions ---

# Prediction 1 (Expected True)
pred1, prob1 = predict_relation(S_true, R_true, model, tokenizer, device)
print("\n--- TEST CASE 1 (Expected: TRUE) ---")
print(f"Sentence: {S_true}")
print(f"Relation: {R_true}")
print(f"Prediction: {'ENTAILED' if pred1 == 1 else 'NOT ENTAILED'} (Confidence: {prob1:.4f})")

# Prediction 2 (Expected False)
pred2, prob2 = predict_relation(S_false, R_false, model, tokenizer, device)
print("\n--- TEST CASE 2 (Expected: FALSE) ---")
print(f"Sentence: {S_false}")
print(f"Relation: {R_false}")
print(f"Prediction: {'ENTAILED' if pred2 == 1 else 'NOT ENTAILED'} (Confidence: {prob2:.4f})")


--- TEST CASE 1 (Expected: TRUE) ---
Sentence: [E1] Wilton Bridge [/E1] was a major crossing of the [E2] River Wye [/E2]
Relation: X crosses Y
Prediction: NOT ENTAILED (Confidence: 0.0060)

--- TEST CASE 2 (Expected: FALSE) ---
Sentence: The [E1] Mona Lisa [/E1] was painted by [E2] Leonardo da Vinci [/E2].
Relation: X mountain range Y
Prediction: NOT ENTAILED (Confidence: 0.0000)


In [56]:
# !zip -r zsbert_model.zip zsbert_model


In [59]:
# -------------------------------
# Dummy Test Dataset
# -------------------------------
test_cases = [
    ("The Sony [E1] Xperia XZ [/E1] Premium is an [E2] Android [/E2] smartphone manufactured and marketed by Sony.","X operating system Y"),
    ("[E1] Golden Gate Bridge [/E1] spans the [E2] San Francisco Bay [/E2].", "X crosses Y"),
    ("The [E1] Eiffel Tower [/E1] is located in [E2] Paris [/E2].", "X located in Y"),
    ("[E1] Wilton Bridge [/E1] was a major crossing of the [E2] River Wye [/E2].", "X crosses Y"),
    ("[E1] Apple Inc. [/E1] was founded in [E2] Cupertino [/E2].", "X located in Y"),
    ("The [E1] Nile River [/E1] flows through [E2] Egypt [/E2].", "X crosses Y"),
    ("[E1] Barack Obama [/E1] was born in [E2] Honolulu [/E2].", "X crosses Y"),
    ("[E1] Mona Lisa [/E1] was painted by [E2] Leonardo da Vinci [/E2].", "X located in Y"),
    ("[E1] Amazon River [/E1] is the longest river in [E2] South America [/E2].", "X crosses Y"),
    ("[E1] Microsoft [/E1] was founded by [E2] Bill Gates [/E2].", "X crosses Y"),
    ("[E1] JFK Airport [/E1] is located in [E2] New York City [/E2].", "X place served by transport hub Y"),
    ("[E1] Heathrow Airport [/E1] connects to [E2] London [/E2].", "X place served by transport hub Y"),
    ("[E1] Mount Everest [/E1] lies between [E2] Nepal [/E2] and China.", "X located in Y"),
    ("[E1] Titanic [/E1] sank in the [E2] Atlantic Ocean [/E2].", "X crosses Y"),
    ("[E1] London Underground [/E1] serves the city of [E2] London [/E2].", "X place served by transport hub Y"),
    ("[E1] Sydney Opera House [/E1] is a landmark in [E2] Sydney [/E2].", "X located in Y"),
]

# -------------------------------
# Run Inference
# -------------------------------
for i, (s, r) in enumerate(test_cases, 1):
    pred, prob = predict_relation(s, r, model, tokenizer, device)
    print(f"\n--- TEST CASE {i} ---")
    print(f"Sentence: {s}")
    print(f"Relation: {r}")
    print(f"Prediction: {'ENTAILED' if pred == 1 else 'NOT ENTAILED'} (Confidence: {prob:.4f})")



--- TEST CASE 1 ---
Sentence: The Sony [E1] Xperia XZ [/E1] Premium is an [E2] Android [/E2] smartphone manufactured and marketed by Sony.
Relation: X operating system Y
Prediction: ENTAILED (Confidence: 0.9999)

--- TEST CASE 2 ---
Sentence: [E1] Golden Gate Bridge [/E1] spans the [E2] San Francisco Bay [/E2].
Relation: X crosses Y
Prediction: NOT ENTAILED (Confidence: 0.0155)

--- TEST CASE 3 ---
Sentence: The [E1] Eiffel Tower [/E1] is located in [E2] Paris [/E2].
Relation: X located in Y
Prediction: ENTAILED (Confidence: 0.9244)

--- TEST CASE 4 ---
Sentence: [E1] Wilton Bridge [/E1] was a major crossing of the [E2] River Wye [/E2].
Relation: X crosses Y
Prediction: NOT ENTAILED (Confidence: 0.0228)

--- TEST CASE 5 ---
Sentence: [E1] Apple Inc. [/E1] was founded in [E2] Cupertino [/E2].
Relation: X located in Y
Prediction: ENTAILED (Confidence: 0.9335)

--- TEST CASE 6 ---
Sentence: The [E1] Nile River [/E1] flows through [E2] Egypt [/E2].
Relation: X crosses Y
Prediction: NOT EN